In [3]:
!pip install groq

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [10]:
import json
import os
import groq
import re

# Set Groq API Key
os.environ["GROQ_API_KEY"] = "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"

# Load Waste Data from JSON File
with open("waste.json", "r") as file:
    waste_data = json.load(file)

# Function to Create Prompt for AI
def create_prompt(waste_info):
    chemical_composition = waste_info.get("composition", "Unknown")
    waste_label = waste_info.get("label", "General Waste")

    prompt = f"""
    The following waste material has the given chemical composition:
    {chemical_composition}
    
    Waste Label: {waste_label}
    
    Task:
    1. Determine if this waste is recyclable or non-recyclable.
    2. Suggest useful products that can be derived from this waste.
    
    Answer in a structured JSON format.
    """
    return prompt

# Function to Extract JSON from AI Response
def extract_json(response_text):
    json_match = re.search(r"\{.*\}", response_text, re.DOTALL)  # Extract JSON part
    if json_match:
        return json_match.group(0)  # Return only the JSON part
    return None

# Initialize Groq API Client
client = groq.Groq(api_key=os.getenv("GROQ_API_KEY"))

# Store AI responses
output_data = []

for waste in waste_data:
    prompt = create_prompt(waste)

    response = client.chat.completions.create(
        model="llama3-8b-8192",  
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0.5,
    )

    # Get AI response
    ai_response = response.choices[0].message.content

    # Extract JSON from AI response
    extracted_json = extract_json(ai_response)

    if extracted_json:
        try:
            response_json = json.loads(extracted_json)  # Convert to valid JSON
            output_data.append(response_json)
        except json.JSONDecodeError:
            print("❌ Failed to decode JSON:", extracted_json)
    else:
        print("❌ No valid JSON found in AI response:", ai_response)

# Save AI Output to JSON File
with open("output.json", "w") as outfile:
    json.dump(output_data, outfile, indent=4)

print("✅ Output stored in output.json")


✅ Output stored in output.json


Testing

In [7]:
import json
import os
import groq
import re  # Import regex for cleaning JSON response

# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"

# Load JSON file
with open("waste.json", "r") as file:
    waste_data = json.load(file)

# Function to create prompt
def create_prompt(waste_info):
    chemical_composition = waste_info.get("composition", "Unknown")
    waste_label = waste_info.get("label", "General Waste")

    prompt = f"""
    You are an AI that strictly outputs only valid JSON. Do not include explanations or extra formatting.
    
    The following waste material has the given chemical composition:
    {chemical_composition}
    
    Waste Label: {waste_label}
    
    Task:
    1. Determine if this waste is recyclable or non-recyclable.
    2. Suggest useful products that can be derived from this waste.
    
    Respond in a structured JSON format only.
    """
    return prompt

# Initialize Groq client
client = groq.Groq(api_key=os.getenv("GROQ_API_KEY"))

# List to store responses
output_data = []

for waste in waste_data:
    prompt = create_prompt(waste)

    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0.5,
    )

    # Extract AI response
    ai_response = response.choices[0].message.content

    # 🔹 **Fix: Remove Markdown Code Blocks (` ``` `)**
    cleaned_json = re.sub(r"```json\n(.*?)\n```", r"\1", ai_response, flags=re.DOTALL)

    # Convert response to JSON
    try:
        structured_response = json.loads(cleaned_json)
        output_data.append(structured_response)
    except json.JSONDecodeError:
        print("⚠️ AI response is not valid JSON, storing raw response instead.")
        output_data.append({"error": "Invalid JSON", "raw_output": ai_response})

# Save the output to a JSON file
with open("output.json", "w") as outfile:
    json.dump(output_data, outfile, indent=4)

print("✅ Output stored in output.json")


✅ Output stored in output.json
